# Set-Up

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
from collections import OrderedDict
L = LinearRegression(n_jobs=-1)
from functools import reduce

# Extract Data

In [2]:
train = pd.read_csv('train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

# Data Research

In [3]:
pd.options.display.max_columns = None
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
missing = dict(train.isnull().sum().sort_values(ascending=False)[:19])

In [5]:
def delete_missing(df):
    '''
    Delete columns with more than 10% missing values
    '''
    for key, value in missing.items():
        missing_num = value/len(train)
        if missing_num > 0.1:
            del df[key]

In [6]:
delete_missing(train)

In [7]:
# Because 'MiscFeature' has many null values, also will delete 'MiscVal'
del train['MiscVal']

In [8]:
train.isnull().sum().sort_values(ascending=False)[:13]

GarageFinish    81
GarageType      81
GarageCond      81
GarageQual      81
GarageYrBlt     81
BsmtExposure    38
BsmtFinType2    38
BsmtFinType1    37
BsmtCond        37
BsmtQual        37
MasVnrType       8
MasVnrArea       8
Electrical       1
dtype: int64

In [9]:
def fill_missing(df):
    df['GarageQual'] = df['GarageQual'].fillna('None')
    df['GarageCond'] = df['GarageCond'].fillna('None')
    df['GarageType'] = df['GarageType'].fillna('None')
    df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0)
    df['GarageFinish'] = df['GarageFinish'].fillna('None')
    df['BsmtExposure'] = df['BsmtExposure'].fillna('None')
    df['BsmtFinType2'] = df['BsmtFinType2'].fillna('None')
    df['BsmtQual'] = df['BsmtQual'].fillna('None')
    df['BsmtFinType1'] = df['BsmtFinType1'].fillna('None')
    df['BsmtCond'] = df['BsmtCond'].fillna('None')
    df['MasVnrType'] = df['MasVnrType'].fillna('None')
    df['MasVnrArea'] = df['MasVnrArea'].fillna(0)
    df = df.dropna()
    return df

In [10]:
train = fill_missing(train)
train.shape

(1459, 74)

## Numeric

Feature Engineering To-Do if needed
* Change to category type?
    * MSSubClass
    * OverallQual
* Convert to datetime
    * MoSold
    * YrSold
    * GarageYrBlt

In [11]:
train.select_dtypes(include='number').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 1459
Data columns (total 36 columns):
Id               1459 non-null int64
MSSubClass       1459 non-null int64
LotArea          1459 non-null int64
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
YearBuilt        1459 non-null int64
YearRemodAdd     1459 non-null int64
MasVnrArea       1459 non-null float64
BsmtFinSF1       1459 non-null int64
BsmtFinSF2       1459 non-null int64
BsmtUnfSF        1459 non-null int64
TotalBsmtSF      1459 non-null int64
1stFlrSF         1459 non-null int64
2ndFlrSF         1459 non-null int64
LowQualFinSF     1459 non-null int64
GrLivArea        1459 non-null int64
BsmtFullBath     1459 non-null int64
BsmtHalfBath     1459 non-null int64
FullBath         1459 non-null int64
HalfBath         1459 non-null int64
BedroomAbvGr     1459 non-null int64
KitchenAbvGr     1459 non-null int64
TotRmsAbvGrd     1459 non-null int64
Fireplaces       1459 non-null int64
Gar

Consolidate the following features:
* BsmtFullBath and BsmtHalfBath to get BsmtBath
* FullBath and HalfBath to get Baths
* All Porch types to retrieve PorchSF

In [12]:
def consolidate(df):
    ''' 
    Consolidate features into one to capture BsmtBath, Baths, and PorchSF
    '''
    df['BsmtBath'] = df['BsmtFullBath'] + df['BsmtHalfBath'].apply(lambda x: x * 0.5)
    df['Baths'] = df['FullBath'] + df['HalfBath'].apply(lambda x : x * 0.5)
    df['PorchSF'] = df['WoodDeckSF'] + df['OpenPorchSF'] + df['EnclosedPorch'] + df['3SsnPorch'] + df['ScreenPorch']
    df = df.drop(['Id','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch'], axis = 1)
    return df

In [13]:
train = consolidate(train)

In [14]:
train.select_dtypes(include='number').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 1459
Data columns (total 29 columns):
MSSubClass      1459 non-null int64
LotArea         1459 non-null int64
OverallQual     1459 non-null int64
OverallCond     1459 non-null int64
YearBuilt       1459 non-null int64
YearRemodAdd    1459 non-null int64
MasVnrArea      1459 non-null float64
BsmtFinSF1      1459 non-null int64
BsmtFinSF2      1459 non-null int64
BsmtUnfSF       1459 non-null int64
TotalBsmtSF     1459 non-null int64
1stFlrSF        1459 non-null int64
2ndFlrSF        1459 non-null int64
LowQualFinSF    1459 non-null int64
GrLivArea       1459 non-null int64
BedroomAbvGr    1459 non-null int64
KitchenAbvGr    1459 non-null int64
TotRmsAbvGrd    1459 non-null int64
Fireplaces      1459 non-null int64
GarageYrBlt     1459 non-null float64
GarageCars      1459 non-null int64
GarageArea      1459 non-null int64
PoolArea        1459 non-null int64
MoSold          1459 non-null int64
YrSold          1459 non-

## String

In [15]:
train.select_dtypes(exclude='number').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 1459
Data columns (total 38 columns):
MSZoning         1459 non-null object
Street           1459 non-null object
LotShape         1459 non-null object
LandContour      1459 non-null object
Utilities        1459 non-null object
LotConfig        1459 non-null object
LandSlope        1459 non-null object
Neighborhood     1459 non-null object
Condition1       1459 non-null object
Condition2       1459 non-null object
BldgType         1459 non-null object
HouseStyle       1459 non-null object
RoofStyle        1459 non-null object
RoofMatl         1459 non-null object
Exterior1st      1459 non-null object
Exterior2nd      1459 non-null object
MasVnrType       1459 non-null object
ExterQual        1459 non-null object
ExterCond        1459 non-null object
Foundation       1459 non-null object
BsmtQual         1459 non-null object
BsmtCond         1459 non-null object
BsmtExposure     1459 non-null object
BsmtFinType1     14

In [16]:
string_columns = train.select_dtypes(exclude='number').columns

In [17]:
for i in string_columns:
    print(train[i].unique())

['RL' 'RM' 'C (all)' 'FV' 'RH']
['Pave' 'Grvl']
['Reg' 'IR1' 'IR2' 'IR3']
['Lvl' 'Bnk' 'Low' 'HLS']
['AllPub' 'NoSeWa']
['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']
['Gtl' 'Mod' 'Sev']
['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'NWAmes'
 'OldTown' 'BrkSide' 'Sawyer' 'NridgHt' 'NAmes' 'SawyerW' 'IDOTRR'
 'MeadowV' 'Edwards' 'Timber' 'Gilbert' 'StoneBr' 'ClearCr' 'NPkVill'
 'Blmngtn' 'BrDale' 'SWISU' 'Blueste']
['Norm' 'Feedr' 'PosN' 'Artery' 'RRAe' 'RRNn' 'RRAn' 'PosA' 'RRNe']
['Norm' 'Artery' 'RRNn' 'Feedr' 'PosN' 'PosA' 'RRAn' 'RRAe']
['1Fam' '2fmCon' 'Duplex' 'TwnhsE' 'Twnhs']
['2Story' '1Story' '1.5Fin' '1.5Unf' 'SFoyer' 'SLvl' '2.5Unf' '2.5Fin']
['Gable' 'Hip' 'Gambrel' 'Mansard' 'Flat' 'Shed']
['CompShg' 'WdShngl' 'Metal' 'WdShake' 'Membran' 'Tar&Grv' 'Roll'
 'ClyTile']
['VinylSd' 'MetalSd' 'Wd Sdng' 'HdBoard' 'BrkFace' 'WdShing' 'CemntBd'
 'Plywood' 'AsbShng' 'Stucco' 'BrkComm' 'AsphShn' 'Stone' 'ImStucc'
 'CBlock']
['VinylSd' 'MetalSd' 'Wd Shng' 'HdBoard' 'Plywood

In [32]:
def convert_qual(df, columns):
    '''
    Convert category quality and conditions to ordinal values
    '''
    for i in columns:
        df[i][df[i] == 'None'] = 0
        df[i][df[i] == 'Po'] = 1
        df[i][df[i] == 'Fa'] = 2
        df[i][df[i] == 'TA'] = 3
        df[i][df[i] == 'Gd'] = 4
        df[i][df[i] == 'Ex'] = 5
        df[i] = df[i].astype('float64')
    return df

In [33]:
train = convert_qual(train, ['ExterQual','ExterCond','BsmtQual','BsmtCond','HeatingQC','KitchenQual',
                            'GarageQual','GarageCond'])

/Users/thisbekristin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/thisbekristin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/thisbekristin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org

In [41]:
def convert_misc_qual(tr):
    tr.LotShape[tr['LotShape']=='IR3'] = 1
    tr.LotShape[tr['LotShape']=='IR2'] = 2
    tr.LotShape[tr['LotShape']=='IR1'] = 3
    tr.LotShape[tr['LotShape']=='Reg'] = 4
    tr.LotShape = tr.LotShape.astype('float64')
    tr.BsmtExposure[tr['BsmtExposure']=='None'] = 0
    tr.BsmtExposure[tr['BsmtExposure']=='No'] = 1
    tr.BsmtExposure[tr['BsmtExposure']=='Mn'] = 2
    tr.BsmtExposure[tr['BsmtExposure']=='Av'] = 3
    tr.BsmtExposure[tr['BsmtExposure']=='Gd'] = 4
    tr.BsmtExposure = tr.BsmtExposure.astype('float64')
    tr.BsmtFinType1[tr['BsmtFinType1']=='None'] = 0
    tr.BsmtFinType1[tr['BsmtFinType1']=='Unf'] = 1
    tr.BsmtFinType1[tr['BsmtFinType1']=='LwQ'] = 2
    tr.BsmtFinType1[tr['BsmtFinType1']=='Rec'] = 3
    tr.BsmtFinType1[tr['BsmtFinType1']=='BLQ'] = 4
    tr.BsmtFinType1[tr['BsmtFinType1']=='ALQ'] = 5
    tr.BsmtFinType1[tr['BsmtFinType1']=='GLQ'] = 6
    tr.BsmtFinType1 = tr.BsmtFinType1.astype('float64')
    tr.BsmtFinType2[tr['BsmtFinType2']=='None'] = 0
    tr.BsmtFinType2[tr['BsmtFinType2']=='Unf'] = 1
    tr.BsmtFinType2[tr['BsmtFinType2']=='LwQ'] = 2
    tr.BsmtFinType2[tr['BsmtFinType2']=='Rec'] = 3
    tr.BsmtFinType2[tr['BsmtFinType2']=='BLQ'] = 4
    tr.BsmtFinType2[tr['BsmtFinType2']=='ALQ'] = 5
    tr.BsmtFinType2[tr['BsmtFinType2']=='GLQ'] = 6
    tr.BsmtFinType2 = tr.BsmtFinType2.astype('float64')
    tr.Functional[tr['Functional']=='None'] = 0
    tr.Functional[tr['Functional']=='Sal'] = 1
    tr.Functional[tr['Functional']=='Sev'] = 2
    tr.Functional[tr['Functional']=='Maj2'] = 3
    tr.Functional[tr['Functional']=='Maj1'] = 4
    tr.Functional[tr['Functional']=='Mod'] = 5
    tr.Functional[tr['Functional']=='Min2'] = 6
    tr.Functional[tr['Functional']=='Min1'] = 7
    tr.Functional[tr['Functional']=='Typ'] = 8
    tr.Functional = tr.Functional.astype('float64')
    tr.GarageFinish[tr['GarageFinish']=='None'] = 0
    tr.GarageFinish[tr['GarageFinish']=='Unf'] = 1
    tr.GarageFinish[tr['GarageFinish']=='RFn'] = 2
    tr.GarageFinish[tr['GarageFinish']=='Fin'] = 3
    tr.GarageFinish = tr.GarageFinish.astype('float64')
    return tr

In [42]:
train = convert_misc_qual(train)

/Users/thisbekristin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/thisbekristin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/thisbekristin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

### 67 Columns after Feature Engineering
* Consolidate
* Delete